In [ ]:
# ATTENTION: only run this cell when on google colab
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Change the current directory to the "Eon" folder
%cd /content/drive/MyDrive/Eon

Mounted at /content/drive
/content/drive/MyDrive/Eon


In [ ]:
#!git clone https://github.com/philshiu/Drosophila_brain_model.git
pip install brian2
%cd Drosophila_brain_model

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.1 MB/s eta 0:00:00
/content/drive/MyDrive/Eon/Drosophila_brain_model


In [ ]:
from model import run_exp
from model import default_params as params
import utils as utl
from brian2 import Hz

config = {
    'path_res'  : './results/new',                 # directory to store results
    'path_comp' : './Completeness_783.csv',        # csv of the complete list of Flywire neurons
    'path_con'  : './Connectivity_783.parquet',    # connectivity data
    'n_proc'    : -1,                              # number of CPU cores (-1: use all)
}

List all of our output neurons.

In [ ]:
flyid2name = { }
#P9_oDN1 corresponds to forward velocity
P9_oDN1_left = 720575940626730883
P9_oDN1_right = 720575940620300308
P9_left = 720575940627652358
P9_right =720575940635872101
#DNa01 and DNa02 correspond to turning
DNa01_right = 720575940627787609
DNa01_left = 720575940644438551
DNa02_right = 720575940629327659
DNa02_left = 720575940604737708
#MDN, "Moonwalker descending neurons" are backwards walking/escape/startle response
MDN_1, MDN_2, MDN_3, MDN_4 = 720575940616026939, 720575940631082808,720575940640331472,720575940610236514
#Giant Fiber corresponds to escape
Giant_Fiber_1, Giant_Fiber_2 = 720575940622838154,720575940632499757
#MN9 corresponds to proboscis motor neuron, and corresponds to feeding.
MN9_left = 720575940660219265
MN9_right = 720575940618238523
#aDN1 correspond to antennal grooming
aDN1_right = 720575940616185531
aDN1_left = 720575940624319124


flyid2name[P9_oDN1_left]=       "P9_oDN1_left"
flyid2name[P9_oDN1_right]=      "P9_oDN1_right"
flyid2name[P9_left]=    "P9_left"
flyid2name[P9_right]=   "P9_right"
flyid2name[DNa01_right]=        "DNa01_right"
flyid2name[DNa01_left]= "DNa01_left"
flyid2name[DNa02_right]=        "DNa02_right"
flyid2name[DNa02_left]= "DNa02_left"
flyid2name[MDN_1]= "MDN_1"
flyid2name[MDN_2]= "MDN_2"
flyid2name[MDN_3]= "MDN_3"
flyid2name[MDN_4]= "MDN_4"
flyid2name[Giant_Fiber_1]= "Giant_Fiber_1"
flyid2name[Giant_Fiber_2]= "Giant_Fiber_2"
flyid2name[MN9_left]= "MN9_left"
flyid2name[MN9_right]= "MN9_right"
flyid2name[aDN1_right]= "aDN1_right"
flyid2name[aDN1_left]= "aDN1_left"

In [ ]:
output_neurons = [P9_oDN1_left, P9_oDN1_right, DNa01_right, DNa01_left, DNa02_right, DNa02_left, MDN_1, MDN_2, MDN_3, MDN_4, Giant_Fiber_1, Giant_Fiber_2, MN9_left, MN9_right, aDN1_right, aDN1_left]

In [ ]:
P9s = [P9_left, P9_right]

First, run P9s to simulate just forward walking.

In [ ]:
# run with different frequency
P9s = [P9_left, P9_right]
params['r_poi'] = 100 * Hz
run_exp(exp_name='P9s_100Hz', neu_exc=P9s, params=params, **config)

>>> Experiment:     P9s_100Hz
    Output file:    results/new/P9s_100Hz.parquet
    Excited neurons: 2


WARNING    /usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]
  warnings.warn(



    Elapsed time:   551 s


In [ ]:
ps = [    './results/new/P9s_100Hz.parquet',]
df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate.sort_values('P9s_100Hz', ascending=False, inplace=True)
df_rate


exp_name,name,P9s_100Hz
flyid,,
720575940627652358,P9_left,96.133333
720575940635872101,P9_right,94.600000
720575940613626521,,37.933333
720575940633901395,,33.033333
720575940631924584,,25.133333
...,...,...
720575940633799787,,0.033333
720575940635170484,,0.033333
720575940633996883,,0.033333


In [ ]:
relevant_neurons = [neuron for neuron in output_neurons if neuron in df_rate.index]
display(df_rate.loc[relevant_neurons])

exp_name,name,P9s_100Hz
flyid,,
720575940626730883,P9_oDN1_left,14.833333
720575940620300308,P9_oDN1_right,10.733333
720575940629327659,DNa02_right,6.433333
720575940604737708,DNa02_left,17.033333


Activate P9s and leg sugar-sensing neurons

In [ ]:
#These neurons correspond to leg sugar-sensing neurons
lgAG2 = [
    720575940627903804,
    720575940606567125,
    720575940617597268,
    720575940633366498,
    720575940622861326,
    720575940624967305,
    720575940625526805,
    720575940612446866,
    720575940622450924,
    720575940625632516,
    720575940627588218,
    720575940633189624,

]
leg_ascending_neurons = [
    720575940627697664, 720575940618066369
]
leg_sugar_sensing_neurons = lgAG2 + leg_ascending_neurons

In [ ]:
# Add leg sugar-sensing neurons to flyid2name
for i, neuron_id in enumerate(lgAG2):
    flyid2name[neuron_id] = f"lgAG2_{i+1}"

for i, neuron_id in enumerate(leg_ascending_neurons):
    flyid2name[neuron_id] = f"leg_ascending_neuron_{i+1}"

In [ ]:
Leg_freq=200
params['r_poi'] = 100 * Hz
params['r_poi2'] = Leg_freq * Hz
run_exp(exp_name='P9_legs', neu_exc=P9s, neu_exc2=leg_sugar_sensing_neurons, params=params, **config)

>>> Experiment:     P9_legs
    Output file:    results/new/P9_legs.parquet
    Excited neurons: 16


WARNING    /usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]
  warnings.warn(



    Elapsed time:   1012 s


In [ ]:
ps = [
    './results/new/P9s_100Hz.parquet',
    './results/new/P9_legs.parquet',
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate.sort_values('P9s_100Hz', ascending=False, inplace=True)
df_rate


exp_name,name,P9_legs,P9s_100Hz
flyid,,,
720575940627652358,P9_left,98.500000,96.133333
720575940635872101,P9_right,98.133333,94.600000
720575940613626521,,38.533333,37.933333
720575940633901395,,53.466667,33.033333
720575940631924584,,26.433333,25.133333
...,...,...,...
720575940655011489,,45.833333,NaN
720575940655013281,,0.033333,NaN
720575940660219265,MN9_left,0.533333,NaN


In [ ]:
relevant_neurons = [neuron for neuron in output_neurons if neuron in df_rate.index]
display(df_rate.loc[relevant_neurons])

exp_name,name,P9_legs,P9s_100Hz
flyid,,,
720575940626730883,P9_oDN1_left,8.800000,14.833333
720575940620300308,P9_oDN1_right,8.633333,10.733333
720575940629327659,DNa02_right,22.466667,6.433333
720575940604737708,DNa02_left,20.933333,17.033333
720575940610236514,MDN_4,0.200000,NaN
720575940660219265,MN9_left,0.533333,NaN


Just activate labellar sugar GRNs.

In [ ]:
sugar_GRNs = [720575940616885538,720575940630233916,720575940639332736,720575940632889389,720575940617000768,720575940632425919,720575940637568838,720575940629176663,720575940621502051,720575940638202345,720575940612670570,720575940611875570,720575940621754367,720575940633143833,720575940613601698,720575940630797113,720575940639198653,720575940639259967,720575940624963786,720575940640649691,720575940610788069,720575940623172843,720575940628853239]

In [ ]:
params['r_poi'] = 200 * Hz
run_exp(exp_name='Sugar_200Hz', neu_exc=sugar_GRNs, params=params, **config)

>>> Experiment:     Sugar_200Hz
    Output file:    results/new/Sugar_200Hz.parquet
    Excited neurons: 23


WARNING    /usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]
  warnings.warn(



    Elapsed time:   1023 s


In [ ]:
ps = [
    './results/new/P9s_100Hz.parquet',
    './results/new/P9_legs.parquet',
    './results/new/Sugar_200Hz.parquet',
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate.sort_values('P9s_100Hz', ascending=False, inplace=True)
df_rate


exp_name,name,P9_legs,P9s_100Hz,Sugar_200Hz
flyid,,,,
720575940627652358,P9_left,98.500000,96.133333,NaN
720575940635872101,P9_right,98.133333,94.600000,NaN
720575940613626521,,38.533333,37.933333,NaN
720575940633901395,,53.466667,33.033333,NaN
720575940631924584,,26.433333,25.133333,NaN
...,...,...,...,...
720575940660220289,,NaN,NaN,4.733333
720575940660223873,,1.700000,NaN,62.566667
720575940660224641,,NaN,NaN,3.000000


In [ ]:
relevant_neurons = [neuron for neuron in output_neurons if neuron in df_rate.index]
display(df_rate.loc[relevant_neurons])

exp_name,name,P9_legs,P9s_100Hz,Sugar_200Hz
flyid,,,,
720575940626730883,P9_oDN1_left,8.800000,14.833333,NaN
720575940620300308,P9_oDN1_right,8.633333,10.733333,NaN
720575940629327659,DNa02_right,22.466667,6.433333,NaN
720575940604737708,DNa02_left,20.933333,17.033333,NaN
720575940610236514,MDN_4,0.200000,NaN,NaN
720575940660219265,MN9_left,0.533333,NaN,93.133333
720575940618238523,MN9_right,NaN,NaN,62.733333


Now add bitter!

In [ ]:
bitter_GRNs = [720575940619072513,720575940646212996,720575940622298631,720575940642088333,720575940627692048,720575940617239197,720575940618682526,
               720575940604714528,720575940603266592,720575940604027168,720575940619197093,720575940610259370,720575940627578156,720575940629481516,
               720575940618887217,720575940614281266,720575940634859188,720575940645743412,720575940637742911,720575940617094208,720575940629416318,
               720575940630195909,720575940615641798,720575940638312262,720575940624310345,720575940621778381,720575940619659861,720575940629146711,
               720575940625750105,720575940610483162,720575940610481370,720575940602353632,720575940610773090,720575940617433830,720575940628962407,
               720575940626287336,720575940623183083,720575940618025199,720575940619028208,720575940621864060,720575940613061118,720575940621008895,
               ]

In [ ]:
for i, neuron_id in enumerate(bitter_GRNs):
    flyid2name[neuron_id] = f"bitter_GRN_{i+1}"

In [ ]:
Activation_freq=200
params['r_poi'] = 200 * Hz
params['r_poi2'] = Activation_freq * Hz
run_exp(exp_name='Sugar_and_bitter', neu_exc=sugar_GRNs, neu_exc2=bitter_GRNs, params=params, **config)

>>> Experiment:     Sugar_and_bitter
    Output file:    results/new/Sugar_and_bitter.parquet
    Excited neurons: 65


WARNING    /usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]
  warnings.warn(



    Elapsed time:   1682 s


In [ ]:
ps = [
    './results/new/P9s_100Hz.parquet',
    './results/new/P9_legs.parquet',
    './results/new/Sugar_200Hz.parquet',
    './results/new/Sugar_and_bitter.parquet',
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate.sort_values('P9s_100Hz', ascending=False, inplace=True)
df_rate


exp_name,name,P9_legs,P9s_100Hz,Sugar_200Hz,Sugar_and_bitter
flyid,,,,,
720575940627652358,P9_left,98.500000,96.133333,NaN,NaN
720575940635872101,P9_right,98.133333,94.600000,NaN,NaN
720575940613626521,,38.533333,37.933333,NaN,NaN
720575940633901395,,53.466667,33.033333,NaN,NaN
720575940631924584,,26.433333,25.133333,NaN,NaN
...,...,...,...,...,...
720575940660223873,,1.700000,NaN,62.566667,2.066667
720575940660224641,,NaN,NaN,3.000000,NaN
720575940660229505,,NaN,NaN,45.533333,24.866667


In [ ]:
relevant_neurons = [neuron for neuron in output_neurons if neuron in df_rate.index]
display(df_rate.loc[relevant_neurons])

exp_name,name,P9_legs,P9s_100Hz,Sugar_200Hz,Sugar_and_bitter
flyid,,,,,
720575940626730883,P9_oDN1_left,8.800000,14.833333,NaN,NaN
720575940620300308,P9_oDN1_right,8.633333,10.733333,NaN,NaN
720575940629327659,DNa02_right,22.466667,6.433333,NaN,NaN
720575940604737708,DNa02_left,20.933333,17.033333,NaN,NaN
720575940610236514,MDN_4,0.200000,NaN,NaN,NaN
720575940660219265,MN9_left,0.533333,NaN,93.133333,0.633333
720575940618238523,MN9_right,NaN,NaN,62.733333,0.033333


#Try unilateral ascending sugar activation

In [ ]:
ascending_leg_right = [720575940627697664]

In [ ]:
Activation_freq=200
params['r_poi'] = 100 * Hz
params['r_poi2'] = Activation_freq * Hz
run_exp(exp_name='P9_right_leg', neu_exc=P9s, neu_exc2=ascending_leg_right, params=params, **config)

>>> Experiment:     P9_right_leg
    Output file:    results/new/P9_right_leg.parquet
    Excited neurons: 3


WARNING    /usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]
  warnings.warn(



    Elapsed time:   541 s


In [ ]:
ascending_leg_left = [720575940618066369]
Activation_freq=200
params['r_poi'] = 100 * Hz
params['r_poi2'] = Activation_freq * Hz
run_exp(exp_name='P9_left_leg', neu_exc=P9s, neu_exc2=ascending_leg_left, params=params, **config)

>>> Experiment:     P9_left_leg
    Output file:    results/new/P9_left_leg.parquet
    Excited neurons: 3


WARNING    /usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]
  warnings.warn(



    Elapsed time:   445 s


In [ ]:
ps = [
    './results/new/P9s_100Hz.parquet',
    './results/new/P9_legs.parquet',
    './results/new/P9_right_leg.parquet',
    './results/new/P9_left_leg.parquet',
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate.sort_values('P9s_100Hz', ascending=False, inplace=True)
df_rate


exp_name,name,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,
720575940627652358,P9_left,100.333333,98.500000,98.600000,96.133333
720575940635872101,P9_right,98.500000,98.133333,98.766667,94.600000
720575940613626521,,38.466667,38.533333,39.166667,37.933333
720575940633901395,,48.466667,53.466667,35.433333,33.033333
720575940631924584,,27.033333,26.433333,26.433333,25.133333
...,...,...,...,...,...
720575940655011489,,NaN,45.833333,1.833333,NaN
720575940655013281,,0.066667,0.033333,NaN,NaN
720575940660219265,MN9_left,NaN,0.533333,0.266667,NaN


In [ ]:
relevant_neurons = [neuron for neuron in output_neurons if neuron in df_rate.index]
display(df_rate.loc[relevant_neurons])

exp_name,name,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,
720575940626730883,P9_oDN1_left,16.766667,8.800000,6.133333,14.833333
720575940620300308,P9_oDN1_right,12.066667,8.633333,11.733333,10.733333
720575940629327659,DNa02_right,6.333333,22.466667,30.033333,6.433333
720575940604737708,DNa02_left,28.966667,20.933333,11.166667,17.033333
720575940610236514,MDN_4,NaN,0.200000,NaN,NaN
720575940660219265,MN9_left,NaN,0.533333,0.266667,NaN


#Vision: Looming → Escape

In [ ]:
LC_4s =[720575940605598892,720575940611134833,720575940612580977,720575940613256863,720575940613260959,720575940614914107,720575940615462587,720575940617176321,720575940617266722,720575940618807105,720575940620795728,720575940622108001,720575940624017251,720575940625038090,720575940625934973,720575940625991043,720575940626605200,720575940626626895,720575940628454522,720575940628462340,720575940630851036,720575940638496720,720575940603637438,720575940610522009,720575940612093351,720575940612323025,720575940612380723,720575940612498129,720575940612518055,720575940612968421,720575940613609484,720575940613638041,720575940614572742,720575940614582946,720575940615053580,720575940615127227,720575940615232217,720575940615575007,720575940616066705,720575940616713355,720575940617026260,720575940617348379,720575940618002644,720575940618234704,720575940618234715,720575940618266459,720575940618267227,720575940618275520,720575940618312606,720575940618676440,720575940618709158,720575940618723749,720575940619397542,720575940620314221,720575940620314612,720575940620731380,720575940620903551,720575940621145821,720575940621522458,720575940621753579,720575940622330582,720575940622531767,720575940622939836,720575940624111763,720575940624790781,720575940624856762,720575940625841351,720575940625845447,720575940625906702,720575940625932421,720575940626553596,720575940626916936,720575940627519107,720575940628064260,720575940628081541,720575940628419527,720575940628518400,720575940628599895,720575940628606713,720575940628699560,720575940628891863,720575940629753807,720575940629964591,720575940630154660,720575940630484495,720575940630998339,720575940631032657,720575940631338271,720575940632475449,720575940632715234,720575940632769180,720575940633013355,720575940633218863,720575940633580384,720575940634517856,720575940635835967,720575940636957006,720575940638456227,720575940639817947,720575940640612480,720575940641213824,720575940645821316,720575940649229433,720575940652611745]

In [ ]:
Activation_freq=200
params['r_poi'] = 100 * Hz
params['r_poi2'] = Activation_freq * Hz
run_exp(exp_name='P9_LC4s', neu_exc=P9s, neu_exc2=LC_4s, params=params, **config)

>>> Experiment:     P9_LC4s
    Output file:    results/new/P9_LC4s.parquet
    Excited neurons: 106


WARNING    /usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]
  warnings.warn(



    Elapsed time:   2314 s


In [ ]:
ps = [
    './results/new/P9s_100Hz.parquet',
    './results/new/P9_legs.parquet',
    './results/new/P9_right_leg.parquet',
    './results/new/P9_left_leg.parquet',
    './results/new/P9_LC4s.parquet',
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate.sort_values('P9s_100Hz', ascending=False, inplace=True)
df_rate


exp_name,name,P9_LC4s,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,,
720575940627652358,P9_left,100.833333,100.333333,98.500000,98.600000,96.133333
720575940635872101,P9_right,97.766667,98.500000,98.133333,98.766667,94.600000
720575940613626521,,23.766667,38.466667,38.533333,39.166667,37.933333
720575940633901395,,36.400000,48.466667,53.466667,35.433333,33.033333
720575940631924584,,21.066667,27.033333,26.433333,26.433333,25.133333
...,...,...,...,...,...,...
720575940660219265,MN9_left,NaN,NaN,0.533333,0.266667,NaN
720575940660223873,,NaN,NaN,1.700000,NaN,NaN
720575940660233601,,2.233333,NaN,NaN,NaN,NaN


In [ ]:
relevant_neurons = [neuron for neuron in output_neurons if neuron in df_rate.index]
display(df_rate.loc[relevant_neurons])

exp_name,name,P9_LC4s,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,,
720575940626730883,P9_oDN1_left,0.300000,16.766667,8.800000,6.133333,14.833333
720575940620300308,P9_oDN1_right,0.166667,12.066667,8.633333,11.733333,10.733333
720575940627787609,DNa01_right,10.233333,NaN,NaN,NaN,NaN
720575940644438551,DNa01_left,10.266667,NaN,NaN,NaN,NaN
720575940629327659,DNa02_right,2.233333,6.333333,22.466667,30.033333,6.433333
720575940604737708,DNa02_left,0.266667,28.966667,20.933333,11.166667,17.033333
720575940616026939,MDN_1,8.200000,NaN,NaN,NaN,NaN
720575940631082808,MDN_2,21.600000,NaN,NaN,NaN,NaN
720575940640331472,MDN_3,17.333333,NaN,NaN,NaN,NaN


# Touch: Virtual dust → Grooming and stopping

In [ ]:
JO_C = [720575940625904016,720575940607765259,720575940612217818,720575940614803266,720575940616080212,720575940621887732,720575940623083288,720575940623557158,720575940623991881,720575940624724211,720575940626274314,720575940628218976,720575940630969051,720575940631399008,720575940631482666,720575940632252661,720575940632717988,720575940634609115,720575940635285599,720575940637153648,720575940639405683,720575940642582039,720575940605627424,720575940614908822,720575940632297007,720575940646735668]
JO_E = [720575940605860710,720575940613153320,720575940618898230,720575940619518667,720575940619557937,720575940620293652,720575940621981464,720575940623412360,720575940627542769,720575940628915279,720575940629706496,720575940630458732,720575940631100690,720575940632188497,720575940635028634,720575940639741347,720575940645483555,720575940604879025,720575940604976318,720575940605929161,720575940605997233,720575940606912386,720575940607651083,720575940608369026,720575940608658205,720575940609065173,720575940609221845,720575940609832772,720575940610702968,720575940611000499,720575940611029858,
        720575940611197107,720575940612136666,720575940613142191,720575940613309359,720575940613428945,720575940613521389,720575940613803293,720575940614109615,720575940615276661,720575940615926982,720575940615957563,720575940616162235,720575940616190651,720575940616345049,720575940616479121,720575940616631001,720575940616642363,720575940616661109,720575940616793300,720575940617245908,720575940617341270,720575940617504027,720575940618051485,720575940618293406,720575940618646744,720575940618861873,720575940618896624,720575940618927024,720575940619235486,720575940619235920,720575940619694040,
        720575940619930965,720575940619991232,720575940620055617,720575940620114196,720575940620661675,720575940620791028,720575940621136816,720575940621172545,720575940621244510,720575940621431247,720575940621590132,720575940621639133,720575940622099405,720575940622163020,720575940622202858,720575940622281352,720575940622432726,720575940623001736,720575940623300137,720575940623334891,720575940623406984,720575940623928812,720575940624115785,720575940624137656,720575940624157769,720575940625293084,720575940625489764,720575940626169481,720575940626270236,720575940626524805,720575940626543768,
        720575940626606211,720575940626748232,720575940626753809,720575940627925865,720575940628162300,720575940628210272,720575940628306784,720575940628349446,720575940628355846,720575940628495447,720575940628828688,720575940628908670,720575940629114959,720575940629531095,720575940630094325,720575940630174268,720575940630650130,720575940630715899,720575940630964187,720575940631044059,720575940631191367,720575940631314316,720575940631326751,720575940631367264,720575940631367520,720575940631367776,720575940631368544,720575940631370336,720575940631406944,720575940631412576,720575940631694069,
        720575940631957344,720575940631997580,720575940632154285,720575940632475859,720575940633555425,720575940633810111,720575940634181984,720575940634186091,720575940634419040,720575940634599643,720575940634605531,720575940634640091,720575940634714075,720575940635290231,720575940635345519,720575940635579128,720575940635672677,720575940635959919,720575940636290935,720575940636332663,720575940636395866,720575940636410869,720575940636498907,720575940636768613,720575940637022298,720575940637445183,720575940638064100,720575940638243645,720575940638332494,720575940638431029,720575940638546229,
        720575940638571486,720575940639041443,720575940639233333,720575940639298237,720575940639699059,720575940640651325,720575940640748195,720575940640952027,720575940641656283,720575940651360246,720575940608862677,720575940608986069,720575940611986545,720575940612279731,720575940612286899,720575940612341490,720575940612679594,720575940612841330,720575940614656162,720575940615090582,720575940615239746,720575940615301599,720575940616545110,720575940619218078,720575940619912021,720575940620087888,720575940620537734,720575940620538246,720575940621032513,720575940622683174,720575940623291945,
        720575940624664468,720575940626881625,720575940629534458,720575940633738209,720575940635136880,720575940639161909,720575940639166080,720575940639648573,720575940642309261]
JO_CE = JO_C + JO_E

In [ ]:
JO_CE = [720575940619341105,720575940630122015,720575940611061526,720575940615848788,720575940628444667,720575940627941431,720575940632449619,720575940650244342,720575940631866508,720575940638681845,720575940628978450,720575940609522461,720575940621442224,720575940602506208,720575940629022149,720575940627109991,720575940630020111,720575940615986459,720575940618684481,720575940620382889,720575940630080071,720575940626565455,720575940630319671,720575940602720940,720575940630564179,720575940637632419,720575940615809349,720575940626042149,720575940637054835,720575940602132509,720575940614188149,720575940616951124,720575940628101126,720575940629055721,720575940616589878,720575940622449388,720575940614427195,720575940625797617,720575940638664437,720575940618467195,720575940621729757,720575940613971485,720575940627585688,720575940629650997,720575940630059847,720575940608742409,720575940614351477,720575940633153375,720575940622937528,720575940604753437,720575940611783464,720575940618599872,720575940609541917,720575940637410869,720575940630070343,720575940621397417,720575940614035485,720575940610018266,720575940634634606,720575940614060829,720575940624799290,720575940641921421,720575940623298559,720575940625559358,720575940629138959,720575940621625597,720575940625962568,720575940632767383,720575940624915230]
len(JO_CE)

69

In [ ]:
all_JOs = JO_CE

In [ ]:
Activation_freq=300
params['r_poi'] = 100 * Hz
params['r_poi2'] = Activation_freq * Hz
run_exp(exp_name='P9_JO_CE_2', neu_exc=P9s, neu_exc2=all_JOs, params=params, **config)

>>> Experiment:     P9_JO_CE_2
    Output file:    results/new/P9_JO_CE_2.parquet
    Excited neurons: 71


In [ ]:
ps = [
    './results/new/P9s_100Hz.parquet',
    './results/new/P9_legs.parquet',
    './results/new/P9_right_leg.parquet',
    './results/new/P9_left_leg.parquet',
    './results/new/P9_LC4s.parquet',
    './results/new/P9_JO_CE.parquet',
    './results/new/P9_JO_CE_2.parquet',
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate.sort_values('P9s_100Hz', ascending=False, inplace=True)
df_rate


In [ ]:
relevant_neurons = [neuron for neuron in output_neurons if neuron in df_rate.index]
display(df_rate.loc[relevant_neurons])

exp_name,name,P9_JO_CE,P9_JO_CE_2,P9_LC4s,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,,,,
720575940626730883,P9_oDN1_left,14.433333,33.466667,0.300000,16.766667,8.800000,6.133333,14.833333
720575940620300308,P9_oDN1_right,14.900000,3.833333,0.166667,12.066667,8.633333,11.733333,10.733333
720575940627787609,DNa01_right,NaN,0.033333,10.233333,NaN,NaN,NaN,NaN
720575940644438551,DNa01_left,NaN,NaN,10.266667,NaN,NaN,NaN,NaN
720575940629327659,DNa02_right,7.166667,8.800000,2.233333,6.333333,22.466667,30.033333,6.433333
720575940604737708,DNa02_left,18.033333,20.400000,0.266667,28.966667,20.933333,11.166667,17.033333
720575940616026939,MDN_1,NaN,NaN,8.200000,NaN,NaN,NaN,NaN
720575940631082808,MDN_2,NaN,NaN,21.600000,NaN,NaN,NaN,NaN
720575940640331472,MDN_3,NaN,NaN,17.333333,NaN,NaN,NaN,NaN


In [ ]:
#Something was slightly weird.

In [ ]:
JO_EV = [720575940645106376,720575940615272415,720575940619869120,720575940620257345,720575940620382889,720575940630834683,720575940632449619,720575940634020508,720575940605530302,720575940607140035,720575940608742409,720575940615590843,720575940620410177,
         720575940621870618,720575940622344170,720575940623298559,720575940626042149,720575940627379333,720575940630080071,720575940632128031,
         720575940632307527,720575940634820703,]
JO_EDC=[720575940606154370,720575940605919334,720575940608884931,720575940616655989,720575940620543110,720575940622937528,720575940624799290,
        720575940626565455,720575940627941431,720575940627977457,
        720575940628160617,720575940629188251,720575940641921421]
JO_EDM=[720575940615972027,720575940618941037,720575940619729835,720575940627282279,720575940628903247,720575940604122982,720575940609486690,
        720575940609541917,
        720575940610018266,720575940611061526,720575940611273395,720575940611684787,720575940614060829,720575940616040587,720575940618599872,
        720575940618684481,720575940619663239,720575940619932654,720575940620919578,720575940621218729,720575940622271684,720575940622638276,
        720575940623312828,720575940625797617,720575940625962568,720575940626309438,720575940626666066,720575940627109991,720575940628101126,
        720575940628978450,720575940629055721,720575940629650997,720575940629985900,720575940630992557,720575940637054835,720575940637084762,
        720575940638664437,720575940646927668,720575940646929204,720575940659131009]
JO_EDP=[720575940609522461,720575940610261346,720575940613641915,720575940615469785,720575940616589878,720575940616951124,720575940619479979,
        720575940621218985,720575940628444667,720575940634634606,720575940640753267,720575940650244342]
JO_EVL=[720575940615573597,720575940615848788,720575940619083349,720575940621397417,720575940621625597,720575940622283912,720575940627049731,
        720575940629022149,720575940630122015,720575940630564179,720575940633153375,720575940637410869,720575940638681845,720575940621033477,
        720575940621776410,720575940621815690,720575940622234211,720575940622635817,720575940623897096,720575940626148354,720575940626540821,
        720575940628258715,720575940629743063,720575940630202624,720575940630544967,720575940633553820,720575940644036644]
JO_EVM=[720575940602132509,720575940602506208,720575940610759634,720575940614188149,720575940615809349,720575940615976891,720575940619341105,
        720575940621092534,720575940622419165,720575940622449388,720575940623108134,720575940624981436,720575940628192055,720575940630059847,
        720575940632767383,720575940639296189,720575940645466500,720575940611783464,720575940612307478,720575940612960552,720575940614351477,
        720575940617212134,720575940617434086,720575940618130334,720575940620249734,720575940620940276,720575940621010352,720575940621729757,
        720575940623437547,720575940624546062,720575940624686268,720575940625054647,720575940625605905,720575940626795909,720575940627585688,
        720575940630020111,720575940632175268,720575940634073183,720575940634891700,720575940637012196,720575940637243504,720575940639339392,
        720575940659426177]
JO_EVP=[720575940620444654,720575940631866508,720575940607853833,720575940611088563,720575940612773374,720575940613221928,720575940615024543,720575940615986459,720575940617811013,720575940618467195,720575940621442224,
        720575940622199977,720575940624915230,720575940625559358,720575940627104649,720575940627314088,720575940633058989,720575940636335735]
JO_CA=[720575940605800369,720575940608784579,720575940618135109,720575940626719101,720575940629296185,720575940636137591,720575940602720940,720575940610079857,720575940614427195,720575940616501787,720575940617156445,720575940625909962,720575940626241369,720575940629105658,720575940629138959,720575940636559534,720575940641372661]
JO_CL=[720575940626135548,720575940627751567,720575940604753437,720575940613971485,720575940614835362,720575940623399059,720575940630319671,720575940639082062]
JO_CM=[720575940607386307,720575940634512992,720575940614035485,720575940618901424,720575940630070343,720575940633443353,720575940635058612,720575940637632419,720575940625626000]
all_JOs = JO_EV+JO_EDC+JO_EDM+JO_EDP+JO_EVL+JO_EVM+JO_EVP+JO_CA+JO_CL+JO_CM

In [ ]:
Activation_freq=300
params['r_poi'] = 100 * Hz
params['r_poi2'] = Activation_freq * Hz
run_exp(exp_name='P9_JO_CE_3', neu_exc=P9s, neu_exc2=all_JOs, params=params, **config)

>>> Experiment:     P9_JO_CE_3
    Output file:    results/new/P9_JO_CE_3.parquet
    Excited neurons: 211


WARNING    /usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]
  warnings.warn(



    Elapsed time:   6176 s


In [ ]:
ps = [
    './results/new/P9s_100Hz.parquet',
    './results/new/P9_legs.parquet',
    './results/new/P9_right_leg.parquet',
    './results/new/P9_left_leg.parquet',
    './results/new/P9_LC4s.parquet',
    './results/new/P9_JO_CE.parquet',
    './results/new/P9_JO_CE_2.parquet',
    './results/new/P9_JO_CE_3.parquet',
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate.sort_values('P9s_100Hz', ascending=False, inplace=True)
df_rate


exp_name,name,P9_JO_CE,P9_JO_CE_2,P9_JO_CE_3,P9_LC4s,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,,,,,
720575940627652358,P9_left,97.133333,100.066667,96.800000,100.833333,100.333333,98.500000,98.600000,96.133333
720575940635872101,P9_right,98.433333,98.433333,101.366667,97.766667,98.500000,98.133333,98.766667,94.600000
720575940613626521,,41.166667,29.433333,33.000000,23.766667,38.466667,38.533333,39.166667,37.933333
720575940633901395,,33.866667,32.200000,29.033333,36.400000,48.466667,53.466667,35.433333,33.033333
720575940631924584,,23.466667,50.900000,70.166667,21.066667,27.033333,26.433333,26.433333,25.133333
...,...,...,...,...,...,...,...,...,...
720575940660313729,,NaN,NaN,NaN,NaN,NaN,82.000000,NaN,NaN
720575940660314241,,24.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
720575940660573313,,NaN,NaN,NaN,0.033333,NaN,NaN,NaN,NaN


In [ ]:
relevant_neurons = [neuron for neuron in output_neurons if neuron in df_rate.index]
display(df_rate.loc[relevant_neurons])

exp_name,name,P9_JO_CE,P9_JO_CE_2,P9_JO_CE_3,P9_LC4s,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,,,,,
720575940626730883,P9_oDN1_left,14.433333,33.466667,50.166667,0.300000,16.766667,8.800000,6.133333,14.833333
720575940620300308,P9_oDN1_right,14.900000,3.833333,0.966667,0.166667,12.066667,8.633333,11.733333,10.733333
720575940627787609,DNa01_right,NaN,0.033333,0.033333,10.233333,NaN,NaN,NaN,NaN
720575940644438551,DNa01_left,NaN,NaN,0.533333,10.266667,NaN,NaN,NaN,NaN
720575940629327659,DNa02_right,7.166667,8.800000,19.300000,2.233333,6.333333,22.466667,30.033333,6.433333
720575940604737708,DNa02_left,18.033333,20.400000,26.500000,0.266667,28.966667,20.933333,11.166667,17.033333
720575940616026939,MDN_1,NaN,NaN,NaN,8.200000,NaN,NaN,NaN,NaN
720575940631082808,MDN_2,NaN,NaN,NaN,21.600000,NaN,NaN,NaN,NaN
720575940640331472,MDN_3,NaN,NaN,NaN,17.333333,NaN,NaN,NaN,NaN


#Bilateral JO-CE

In [ ]:
JO_EV = [720575940645106376,720575940615272415,720575940619869120,720575940620257345,720575940620382889,720575940630834683,720575940632449619,720575940634020508,720575940605530302,720575940607140035,720575940608742409,720575940615590843,720575940620410177,
         720575940621870618,720575940622344170,720575940623298559,720575940626042149,720575940627379333,720575940630080071,720575940632128031,
         720575940632307527,720575940634820703,]
JO_EDC=[720575940606154370,720575940605919334,720575940608884931,720575940616655989,720575940620543110,720575940622937528,720575940624799290,
        720575940626565455,720575940627941431,720575940627977457,
        720575940628160617,720575940629188251,720575940641921421]
JO_EDM=[720575940615972027,720575940618941037,720575940619729835,720575940627282279,720575940628903247,720575940604122982,720575940609486690,
        720575940609541917,
        720575940610018266,720575940611061526,720575940611273395,720575940611684787,720575940614060829,720575940616040587,720575940618599872,
        720575940618684481,720575940619663239,720575940619932654,720575940620919578,720575940621218729,720575940622271684,720575940622638276,
        720575940623312828,720575940625797617,720575940625962568,720575940626309438,720575940626666066,720575940627109991,720575940628101126,
        720575940628978450,720575940629055721,720575940629650997,720575940629985900,720575940630992557,720575940637054835,720575940637084762,
        720575940638664437,720575940646927668,720575940646929204,720575940659131009]
JO_EDP=[720575940609522461,720575940610261346,720575940613641915,720575940615469785,720575940616589878,720575940616951124,720575940619479979,
        720575940621218985,720575940628444667,720575940634634606,720575940640753267,720575940650244342]
JO_EVL=[720575940615573597,720575940615848788,720575940619083349,720575940621397417,720575940621625597,720575940622283912,720575940627049731,
        720575940629022149,720575940630122015,720575940630564179,720575940633153375,720575940637410869,720575940638681845,720575940621033477,
        720575940621776410,720575940621815690,720575940622234211,720575940622635817,720575940623897096,720575940626148354,720575940626540821,
        720575940628258715,720575940629743063,720575940630202624,720575940630544967,720575940633553820,720575940644036644]
JO_EVM=[720575940602132509,720575940602506208,720575940610759634,720575940614188149,720575940615809349,720575940615976891,720575940619341105,
        720575940621092534,720575940622419165,720575940622449388,720575940623108134,720575940624981436,720575940628192055,720575940630059847,
        720575940632767383,720575940639296189,720575940645466500,720575940611783464,720575940612307478,720575940612960552,720575940614351477,
        720575940617212134,720575940617434086,720575940618130334,720575940620249734,720575940620940276,720575940621010352,720575940621729757,
        720575940623437547,720575940624546062,720575940624686268,720575940625054647,720575940625605905,720575940626795909,720575940627585688,
        720575940630020111,720575940632175268,720575940634073183,720575940634891700,720575940637012196,720575940637243504,720575940639339392,
        720575940659426177]
JO_EVP=[720575940620444654,720575940631866508,720575940607853833,720575940611088563,720575940612773374,720575940613221928,720575940615024543,720575940615986459,720575940617811013,720575940618467195,720575940621442224,
        720575940622199977,720575940624915230,720575940625559358,720575940627104649,720575940627314088,720575940633058989,720575940636335735]
JO_CA=[720575940605800369,720575940608784579,720575940618135109,720575940626719101,720575940629296185,720575940636137591,720575940602720940,720575940610079857,720575940614427195,720575940616501787,720575940617156445,720575940625909962,720575940626241369,720575940629105658,720575940629138959,720575940636559534,720575940641372661]
JO_CL=[720575940626135548,720575940627751567,720575940604753437,720575940613971485,720575940614835362,720575940623399059,720575940630319671,720575940639082062]
JO_CM=[720575940607386307,720575940634512992,720575940614035485,720575940618901424,720575940630070343,720575940633443353,720575940635058612,720575940637632419,720575940625626000]
all_JOs = JO_EV+JO_EDC+JO_EDM+JO_EDP+JO_EVL+JO_EVM+JO_EVP+JO_CA+JO_CL+JO_CM

In [ ]:
JO_C = [720575940625904016,720575940607765259,720575940612217818,720575940614803266,720575940616080212,720575940621887732,720575940623083288,720575940623557158,720575940623991881,720575940624724211,720575940626274314,720575940628218976,720575940630969051,720575940631399008,720575940631482666,720575940632252661,720575940632717988,720575940634609115,720575940635285599,720575940637153648,720575940639405683,720575940642582039,720575940605627424,720575940614908822,720575940632297007,720575940646735668]
JO_E = [720575940605860710,720575940613153320,720575940618898230,720575940619518667,720575940619557937,720575940620293652,720575940621981464,720575940623412360,720575940627542769,720575940628915279,720575940629706496,720575940630458732,720575940631100690,720575940632188497,720575940635028634,720575940639741347,720575940645483555,720575940604879025,720575940604976318,720575940605929161,720575940605997233,720575940606912386,720575940607651083,720575940608369026,720575940608658205,720575940609065173,720575940609221845,720575940609832772,720575940610702968,720575940611000499,720575940611029858,
        720575940611197107,720575940612136666,720575940613142191,720575940613309359,720575940613428945,720575940613521389,720575940613803293,720575940614109615,720575940615276661,720575940615926982,720575940615957563,720575940616162235,720575940616190651,720575940616345049,720575940616479121,720575940616631001,720575940616642363,720575940616661109,720575940616793300,720575940617245908,720575940617341270,720575940617504027,720575940618051485,720575940618293406,720575940618646744,720575940618861873,720575940618896624,720575940618927024,720575940619235486,720575940619235920,720575940619694040,
        720575940619930965,720575940619991232,720575940620055617,720575940620114196,720575940620661675,720575940620791028,720575940621136816,720575940621172545,720575940621244510,720575940621431247,720575940621590132,720575940621639133,720575940622099405,720575940622163020,720575940622202858,720575940622281352,720575940622432726,720575940623001736,720575940623300137,720575940623334891,720575940623406984,720575940623928812,720575940624115785,720575940624137656,720575940624157769,720575940625293084,720575940625489764,720575940626169481,720575940626270236,720575940626524805,720575940626543768,
        720575940626606211,720575940626748232,720575940626753809,720575940627925865,720575940628162300,720575940628210272,720575940628306784,720575940628349446,720575940628355846,720575940628495447,720575940628828688,720575940628908670,720575940629114959,720575940629531095,720575940630094325,720575940630174268,720575940630650130,720575940630715899,720575940630964187,720575940631044059,720575940631191367,720575940631314316,720575940631326751,720575940631367264,720575940631367520,720575940631367776,720575940631368544,720575940631370336,720575940631406944,720575940631412576,720575940631694069,
        720575940631957344,720575940631997580,720575940632154285,720575940632475859,720575940633555425,720575940633810111,720575940634181984,720575940634186091,720575940634419040,720575940634599643,720575940634605531,720575940634640091,720575940634714075,720575940635290231,720575940635345519,720575940635579128,720575940635672677,720575940635959919,720575940636290935,720575940636332663,720575940636395866,720575940636410869,720575940636498907,720575940636768613,720575940637022298,720575940637445183,720575940638064100,720575940638243645,720575940638332494,720575940638431029,720575940638546229,
        720575940638571486,720575940639041443,720575940639233333,720575940639298237,720575940639699059,720575940640651325,720575940640748195,720575940640952027,720575940641656283,720575940651360246,720575940608862677,720575940608986069,720575940611986545,720575940612279731,720575940612286899,720575940612341490,720575940612679594,720575940612841330,720575940614656162,720575940615090582,720575940615239746,720575940615301599,720575940616545110,720575940619218078,720575940619912021,720575940620087888,720575940620537734,720575940620538246,720575940621032513,720575940622683174,720575940623291945,
        720575940624664468,720575940626881625,720575940629534458,720575940633738209,720575940635136880,720575940639161909,720575940639166080,720575940639648573,720575940642309261]
JO_CE = JO_C + JO_E
all_JOs =all_JOs + JO_CE
print(len(JO_CE))

221


In [ ]:
print(len(all_JOs))

430


In [ ]:
Activation_freq=300
params['r_poi'] = 100 * Hz
params['r_poi2'] = Activation_freq * Hz
run_exp(exp_name='P9_JO_CE_bilateral', neu_exc=P9s, neu_exc2=all_JOs, params=params, **config)

>>> Experiment:     P9_JO_CE_bilateral
    Output file:    results/new/P9_JO_CE_bilateral.parquet
    Excited neurons: 432


WARNING    /usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]
  warnings.warn(



    Elapsed time:   10341 s


In [ ]:
ps = [
    './results/new/P9s_100Hz.parquet',
    './results/new/P9_legs.parquet',
    './results/new/P9_right_leg.parquet',
    './results/new/P9_left_leg.parquet',
    './results/new/P9_LC4s.parquet',
    './results/new/P9_JO_CE.parquet',
    './results/new/P9_JO_CE_2.parquet',
    './results/new/P9_JO_CE_3.parquet',
    './results/new/P9_JO_CE_bilateral.parquet',
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate.sort_values('P9s_100Hz', ascending=False, inplace=True)
df_rate


exp_name,name,P9_JO_CE,P9_JO_CE_2,P9_JO_CE_3,P9_JO_CE_bilateral,P9_LC4s,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,,,,,,
720575940627652358,P9_left,97.133333,100.066667,96.800000,96.800000,100.833333,100.333333,98.500000,98.600000,96.133333
720575940635872101,P9_right,98.433333,98.433333,101.366667,96.266667,97.766667,98.500000,98.133333,98.766667,94.600000
720575940613626521,,41.166667,29.433333,33.000000,43.833333,23.766667,38.466667,38.533333,39.166667,37.933333
720575940633901395,,33.866667,32.200000,29.033333,32.600000,36.400000,48.466667,53.466667,35.433333,33.033333
720575940631924584,,23.466667,50.900000,70.166667,67.333333,21.066667,27.033333,26.433333,26.433333,25.133333
...,...,...,...,...,...,...,...,...,...,...
720575940660313729,,NaN,NaN,NaN,NaN,NaN,NaN,82.000000,NaN,NaN
720575940660314241,,24.833333,NaN,NaN,30.100000,NaN,NaN,NaN,NaN,NaN
720575940660573313,,NaN,NaN,NaN,NaN,0.033333,NaN,NaN,NaN,NaN


In [ ]:
relevant_neurons = [neuron for neuron in output_neurons if neuron in df_rate.index]
display(df_rate.loc[relevant_neurons])

exp_name,name,P9_JO_CE,P9_JO_CE_2,P9_JO_CE_3,P9_JO_CE_bilateral,P9_LC4s,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,,,,,,
720575940626730883,P9_oDN1_left,14.433333,33.466667,50.166667,31.000000,0.300000,16.766667,8.800000,6.133333,14.833333
720575940620300308,P9_oDN1_right,14.900000,3.833333,0.966667,4.366667,0.166667,12.066667,8.633333,11.733333,10.733333
720575940627787609,DNa01_right,NaN,0.033333,0.033333,NaN,10.233333,NaN,NaN,NaN,NaN
720575940644438551,DNa01_left,NaN,NaN,0.533333,0.100000,10.266667,NaN,NaN,NaN,NaN
720575940629327659,DNa02_right,7.166667,8.800000,19.300000,20.500000,2.233333,6.333333,22.466667,30.033333,6.433333
720575940604737708,DNa02_left,18.033333,20.400000,26.500000,22.100000,0.266667,28.966667,20.933333,11.166667,17.033333
720575940616026939,MDN_1,NaN,NaN,NaN,NaN,8.200000,NaN,NaN,NaN,NaN
720575940631082808,MDN_2,NaN,NaN,NaN,NaN,21.600000,NaN,NaN,NaN,NaN
720575940640331472,MDN_3,NaN,NaN,NaN,NaN,17.333333,NaN,NaN,NaN,NaN


# Sound: (startle) → Escape

In [ ]:
JO_A = [720575940627046010,720575940606612937,720575940607367125,720575940617534110,720575940620537435,720575940620649641,720575940620745825,720575940621340429,720575940622591049,720575940624428728,720575940629044071,720575940630017603,720575940630018115,720575940632669741,720575940638256949,720575940607059122,720575940607208114,720575940608457483,720575940608554052,720575940608975371,720575940609691075,720575940610236740,720575940611177586,720575940613304883,720575940613682911,720575940614014614,720575940614280554,720575940614332899,720575940614544582,720575940614866242,720575940614927989,720575940615241010,720575940615676334,720575940617668537,720575940618056753,720575940618770625,720575940618851963,720575940619465131,720575940619508033,720575940619694512,720575940619878485,720575940620322753,720575940621110454,720575940621464344,720575940621791664,720575940621870576,720575940622295585,720575940622706653,720575940623189799,720575940623587780,720575940623840589,720575940624167093,720575940624670759,720575940624770540,720575940625227672,720575940625917386,720575940626162378,720575940626164938,720575940626242191,720575940626522620,720575940626949513,720575940627085199,720575940627106173,720575940627142666,720575940627195691,720575940627442197,720575940627538947,720575940628054760,720575940628356358,720575940628769239,720575940629237036,720575940629450366,720575940630534615,720575940630721910,720575940631156431,720575940631463495,720575940632022888,720575940632812867,720575940633057943,720575940634228124,720575940634414959,720575940637389534,720575940637418549,720575940639375182,720575940640713680,720575940642063245,720575940642163341,720575940643903904,720575940608487772,720575940615041064,720575940615750318,720575940624832231,720575940630211127,720575940639742627]

In [ ]:
JO_B = [720575940632229457,720575940600433181,720575940604531229,720575940606619081,720575940608876471,720575940609934365,720575940612235439,720575940612409885,720575940613127314,720575940616336245,720575940616786315,720575940618856896,720575940621508336,
        720575940622960422,720575940623292925,720575940623357675,720575940623490794,720575940623745869,720575940624594276,720575940624755180,720575940625587970,720575940625910714,720575940627442709,720575940629350394,720575940630630225,720575940630660951,
        720575940630925330,720575940632466784,720575940632692378,720575940634830175,720575940637813220,720575940638636533,720575940638811507,720575940639049560,720575940639145550,720575940639397747,720575940640820788,720575940642123917,720575940644057635,
        720575940604054188,720575940604274622,720575940604805036,720575940605219500,720575940605238188,720575940605755424,720575940606850761,720575940606871497,720575940606872777,720575940606938313,720575940607088514,720575940607827392,720575940607881152,
        720575940607944384,720575940607947456,720575940607948224,720575940608112137,720575940608524224,720575940608875703,720575940608893623,720575940609527069,720575940609816014,720575940610028868,720575940610341525,720575940610595118,720575940610621742,
        720575940610674581,720575940610739666,720575940610753656,720575940611392986,720575940612129962,720575940612355763,720575940612407581,720575940612418483,720575940612419763,720575940612446744,720575940612585577,720575940612647166,720575940612956378,
        720575940613006590,720575940613469545,720575940613622014,720575940613696552,720575940613776962,720575940613924271,720575940613934965,720575940613956767,720575940614205708,720575940614795410,720575940614847906,720575940615222201,720575940615387846,
        720575940615540841,720575940615860061,720575940616413509,720575940616428884,720575940616612025,720575940616653429,720575940616793174,720575940616859622,720575940617066653,720575940617110306,720575940617457881,720575940617517661,
        720575940617541147,720575940617724390,720575940617945246,720575940617972052,720575940618847281,720575940618953270,720575940619447105,720575940619572762,720575940619940811,720575940619941067,720575940619969227,720575940620031341,720575940620386981,
        720575940620422413,720575940620422938,720575940620516846,720575940620575653,720575940620587713,720575940620622785,720575940621077253,720575940621120353,720575940621513566,720575940621722111,720575940621823514,720575940622026757,720575940622342569,
        720575940622344216,720575940622552163,720575940622593476,720575940622857827,720575940622946963,720575940622978823,720575940623146378,720575940623290153,720575940623301417,720575940623401128,720575940623474871,720575940623562534,720575940623805159,
        720575940624065724,720575940624111177,720575940624116553,720575940624562618,720575940625023024,720575940625049945,720575940625624464,720575940625866789,720575940625888560,720575940625972624,720575940626257736,720575940626258532,720575940626693894,
        720575940626726021,720575940626741265,720575940626885960,720575940627249039,720575940627304963,720575940627352595,720575940627437572,720575940627453778,720575940627714310,720575940627839304,720575940627927751,720575940627985351,720575940628159081,
        720575940628315232,720575940628467462,720575940628543473,720575940628698371,720575940628786511,720575940628910235,720575940628957443,720575940629105022,720575940629269630,720575940629275180,720575940629492727,720575940629804414,720575940629806206,
        720575940629860089,720575940630117879,720575940630269945,720575940630461804,720575940630462940,720575940630467804,720575940630785388,720575940630816569,720575940630820347,720575940630910971,720575940630916347,720575940630960325,720575940631448338,
        720575940631461451,720575940631469098,720575940631470378,720575940631699816,720575940631750687,720575940631794513,720575940631915219,720575940632037260,720575940632129567,720575940632288815,720575940632618700,720575940633301217,720575940633524443,
        720575940634178656,720575940634229399,720575940634781594,720575940634892389,720575940635217144,720575940635237727,720575940635278455,720575940635732916,720575940635973989,720575940636276336,720575940636425584,720575940636537663,720575940636990005,
        720575940637612095,720575940637630783,720575940637919551,720575940638191679,720575940639405912,720575940639649152,720575940639745085,720575940639826304,720575940639970392,720575940640130432,720575940640169947,720575940640817204,720575940640836660,
        720575940640841268,720575940640843316,720575940641396078,720575940642108557,720575940642414989,720575940642442893,720575940642897312,720575940643418990,720575940643791304,720575940643888584,720575940643912608,720575940644635172,720575940645085975,
        720575940645086999,720575940645087511,720575940645089559,720575940604988332,720575940605209521,720575940605622118,720575940608483164,720575940609289284,720575940611000274,720575940611154786,720575940612029297,720575940612615833,720575940613783038,
        720575940614644898,720575940614805983,720575940616096306,720575940617244049,720575940617407929,720575940617845505,720575940617938772,720575940620257216,720575940620458735,720575940620661931,720575940620822893,720575940622297855,720575940622998115,
        720575940623529875,720575940624067581,720575940624510868,720575940625565368,720575940627778393,720575940628896299,720575940629253376,720575940629505274,720575940629766782,720575940629989879,720575940630207799,
        720575940631670584,720575940632315473,720575940634257564,720575940634775706,720575940635078239,720575940635345271,720575940636488667,720575940637919295,720575940638081855,720575940638556998,720575940639803453,720575940645539694,720575940646362403]

In [ ]:
JO_AB= JO_A +JO_B

In [ ]:
Activation_freq=200
params['r_poi'] = 100 * Hz
params['r_poi2'] = Activation_freq * Hz
run_exp(exp_name='P9_JO_AB', neu_exc=P9s, neu_exc2=JO_AB, params=params, **config)

>>> Experiment:     P9_JO_AB
    Output file:    results/new/P9_JO_AB.parquet
    Excited neurons: 392


WARNING    /usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]
  warnings.warn(



    Elapsed time:   7773 s


In [ ]:
ps = [
    './results/new/P9s_100Hz.parquet',
    './results/new/P9_legs.parquet',
    './results/new/P9_right_leg.parquet',
    './results/new/P9_left_leg.parquet',
    './results/new/P9_LC4s.parquet',
    './results/new/P9_JO_CE.parquet',
    './results/new/P9_JO_AB.parquet',
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate.sort_values('P9s_100Hz', ascending=False, inplace=True)
df_rate


exp_name,name,P9_JO_AB,P9_JO_CE,P9_LC4s,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,,,,
720575940627652358,P9_left,100.566667,97.133333,100.833333,100.333333,98.500000,98.600000,96.133333
720575940635872101,P9_right,97.433333,98.433333,97.766667,98.500000,98.133333,98.766667,94.600000
720575940613626521,,40.166667,41.166667,23.766667,38.466667,38.533333,39.166667,37.933333
720575940633901395,,34.100000,33.866667,36.400000,48.466667,53.466667,35.433333,33.033333
720575940631924584,,41.100000,23.466667,21.066667,27.033333,26.433333,26.433333,25.133333
...,...,...,...,...,...,...,...,...
720575940660233601,,NaN,NaN,2.233333,NaN,NaN,NaN,NaN
720575940660313729,,NaN,NaN,NaN,NaN,82.000000,NaN,NaN
720575940660314241,,NaN,24.833333,NaN,NaN,NaN,NaN,NaN


In [ ]:
relevant_neurons = [neuron for neuron in output_neurons if neuron in df_rate.index]
display(df_rate.loc[relevant_neurons])

exp_name,name,P9_JO_AB,P9_JO_CE,P9_LC4s,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,,,,
720575940626730883,P9_oDN1_left,12.766667,14.433333,0.300000,16.766667,8.800000,6.133333,14.833333
720575940620300308,P9_oDN1_right,22.966667,14.900000,0.166667,12.066667,8.633333,11.733333,10.733333
720575940627787609,DNa01_right,1.300000,NaN,10.233333,NaN,NaN,NaN,NaN
720575940644438551,DNa01_left,NaN,NaN,10.266667,NaN,NaN,NaN,NaN
720575940629327659,DNa02_right,9.933333,7.166667,2.233333,6.333333,22.466667,30.033333,6.433333
720575940604737708,DNa02_left,17.633333,18.033333,0.266667,28.966667,20.933333,11.166667,17.033333
720575940616026939,MDN_1,NaN,NaN,8.200000,NaN,NaN,NaN,NaN
720575940631082808,MDN_2,NaN,NaN,21.600000,NaN,NaN,NaN,NaN
720575940640331472,MDN_3,NaN,NaN,17.333333,NaN,NaN,NaN,NaN


# Olfaction: Aversive → Repulsion

In [ ]:
Or56a=[720575940659222657,720575940641403021,720575940624211470,720575940616536209,720575940615427734,720575940628380827,720575940654069409,720575940613671330,720575940644590116,720575940612972328,720575940627318696,720575940627805096,720575940632190765,720575940633031085,720575940634955188,720575940621106102,720575940615923131,720575940608928324,720575940631467591,720575940622553420,720575940628086607,720575940626357586,720575940632041043,720575940618946901,720575940616095318,720575940626411097,720575940634614367,720575940603832288,720575940620055905,720575940609633378,720575940637704676,720575940638202852,720575940622713578,720575940635705963,720575940629830508,720575940630257772,720575940619539182,720575940612019442,720575940639931893]

In [ ]:
Or56a_freq=250
params['r_poi'] = 100 * Hz
params['r_poi2'] = Or56a_freq * Hz
run_exp(exp_name='P9_Or56a', neu_exc=P9s, neu_exc2=Or56a, params=params, **config)

>>> Skipping experiment P9_Or56a because results/new/P9_Or56a.parquet exists and force_overwrite = False


In [ ]:
ps = [
    './results/new/P9s_100Hz.parquet',
    './results/new/P9_legs.parquet',
    './results/new/P9_right_leg.parquet',
    './results/new/P9_left_leg.parquet',
    './results/new/P9_LC4s.parquet',
    './results/new/P9_JO_CE.parquet',
    './results/new/P9_JO_AB.parquet',
    './results/new/P9_Or56a.parquet'
]

df_spike = utl.load_exps(ps)
df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyid2name)
df_rate.sort_values('P9s_100Hz', ascending=False, inplace=True)
df_rate


exp_name,name,P9_JO_AB,P9_JO_CE,P9_LC4s,P9_Or56a,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,,,,,
720575940627652358,P9_left,100.566667,97.133333,100.833333,100.633333,100.333333,98.500000,98.600000,96.133333
720575940635872101,P9_right,97.433333,98.433333,97.766667,97.266667,98.500000,98.133333,98.766667,94.600000
720575940613626521,,40.166667,41.166667,23.766667,35.933333,38.466667,38.533333,39.166667,37.933333
720575940633901395,,34.100000,33.866667,36.400000,108.900000,48.466667,53.466667,35.433333,33.033333
720575940631924584,,41.100000,23.466667,21.066667,24.366667,27.033333,26.433333,26.433333,25.133333
...,...,...,...,...,...,...,...,...,...
720575940660742785,,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN
720575940660912001,,NaN,NaN,NaN,54.133333,NaN,NaN,NaN,NaN
720575940660914561,,NaN,NaN,NaN,68.866667,NaN,NaN,NaN,NaN


In [ ]:
relevant_neurons = [neuron for neuron in output_neurons if neuron in df_rate.index]
display(df_rate.loc[relevant_neurons])

exp_name,name,P9_JO_AB,P9_JO_CE,P9_LC4s,P9_Or56a,P9_left_leg,P9_legs,P9_right_leg,P9s_100Hz
flyid,,,,,,,,,
720575940626730883,P9_oDN1_left,12.766667,14.433333,0.300000,0.600000,16.766667,8.800000,6.133333,14.833333
720575940620300308,P9_oDN1_right,22.966667,14.900000,0.166667,1.533333,12.066667,8.633333,11.733333,10.733333
720575940627787609,DNa01_right,1.300000,NaN,10.233333,18.866667,NaN,NaN,NaN,NaN
720575940644438551,DNa01_left,NaN,NaN,10.266667,6.266667,NaN,NaN,NaN,NaN
720575940629327659,DNa02_right,9.933333,7.166667,2.233333,49.633333,6.333333,22.466667,30.033333,6.433333
720575940604737708,DNa02_left,17.633333,18.033333,0.266667,0.966667,28.966667,20.933333,11.166667,17.033333
720575940616026939,MDN_1,NaN,NaN,8.200000,NaN,NaN,NaN,NaN,NaN
720575940631082808,MDN_2,NaN,NaN,21.600000,NaN,NaN,NaN,NaN,NaN
720575940640331472,MDN_3,NaN,NaN,17.333333,NaN,NaN,NaN,NaN,NaN
